In [1]:
import os, re, csv
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
from soyspacing.countbase import CountSpace

In [2]:
def csv2data(*df):
    
    """
    csv 파일은 필수적으로 내용(질문)을 가지고 있어야한다.
    
    """

    df_lst = list(df)
    merged_df_lst = []
    for df in df_lst:
        col = df.columns.tolist()

        if '내용' not in col:
            print("내용이 없습니다")
        else:
            pass

        col_nme = []
        not_need = []
        for name in col:
            if '내용' == name:
                col_nme.append('내용')
                col.remove(name)
            else:
                pass

        for name in col:
            if '댓글' == name:
                col_nme.append('댓글')
                col.remove(name)
            else:
                pass


        df = df.drop(col, axis=1)
        if '댓글' not in df.columns.tolist():
            df['댓글'] = np.nan
        df['TAGSET'] = np.nan
        df.rename(columns={"내용":"QUESTION", "댓글":"ANSWER"}, inplace=True)
        
        merged_df_lst.append(df)

    # 데이터 병합
    merged_df = pd.concat(merged_df_lst)
    merged_df.reset_index(drop=True, inplace=True)
    merged_df = merged_df[['QUESTION', "ANSWER", "TAGSET"]]

    return merged_df

In [3]:
def preprocess(merged_df, space_model_path, stop_word_path, result_file_name):
    """
    import os, re, csv
    import pandas as pd
    import numpy as np
    from kiwipiepy import Kiwi
    from soyspacing.countbase import CountSpace

    pip install soyspace
    pip install kiwipiepy

    model : soy_space_model (자료 폴더에있음)
 model : soy_space_model (자료 폴더에있음)
    stop_word : 자료 파일에 있음

    """
        
    model = CountSpace()
    model.load_model(space_model_path, json_format=False)

    # 질문 리스트화
    question_list = merged_df['QUESTION'].tolist()

    # 띄어쓰기 시작
    soy_corrected_question = []
    for x in question_list:
        corrected = model.correct(x)
        soy_corrected_question.append(corrected)

    spaced_quest = []
    for quest, tag in soy_corrected_question:
        if type(quest) == str:
            spaced_quest.append(quest)
        else:
            continue


    # 정규표현식으로 정규화
    norm_content = []
    for text in spaced_quest:
        punct = re.compile(r'[\~\'\"\.\,\?\!\:\;\)\(\+]+') 
        norm_text = re.sub(punct, "", text)
        norm_text = re.sub(r'(안녕)+(하세요)?', '', norm_text)
        norm_text = re.sub(r'(조교)[님]?', "", norm_text)
        norm_text = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+', '', norm_text) # ㅠ하나 쓰는 놈들꺼 제거...
        norm_text = re.sub( r"\s{2,}"," ", norm_text) 
        norm_text = re.sub( r"\A\s+",'', norm_text)
        norm_text = re.sub(r"([가-힣]+)[여용염]",r"\1요",norm_text) # 용용체 제거 슈바끄...
        norm_text = re.sub(r"\b이중[^전공]*?\b","이중전공",norm_text) # 이중으로만 적혀있는거 변형
        norm_text = re.sub(r"\b부전[^공]*?\b","부전공",norm_text) # 부전으로만 적혀있는거 변형

        # 줄인말제거
        norm_text = re.sub(r"\b전심[가-힣]*\b", "전공심화", norm_text)
        norm_text = re.sub(r"\b융전[가-힣]*\b", "융합전공", norm_text)
        norm_text = re.sub(r'\b연계전공\b', '융합전공', norm_text) # 단어 통일화
        
        norm_content.append(norm_text)

    # 데이터프레임에 병합
    merged_df['QUESTION'] = norm_content

    # 형태소 분석

    kiwi = Kiwi()

    #대분류 사전 추가
    kiwi.add_user_word('이중전공', 'DM', 20.0)
    kiwi.add_user_word('융합전공', 'UM', 20.0)
    kiwi.add_user_word('전공심화', 'MD', 20.0)
    kiwi.add_user_word('부전공', 'SM', 20.0)
    kiwi.add_user_word('연계전공', 'UM', 20.0)

    kiwi.prepare()

    ## 형태소 분석
    sent = ''
    lem_cont = []
    for quest in norm_content:
        ## 형태소만 떼어내기
        temp_tagging = [x[0] for x in kiwi.analyze(quest, top_n=1)]
        inner_temp = ["{}/{}".format(word, tag) for word, tag, score1, score2 in temp_tagging[0]]
        lem_cont.append(inner_temp)

    ## 불용어 처리 ##

    stop_file = open(stop_word_path,'r', encoding = 'utf8')
    lines = stop_file.readlines()
    stop_file.close()
    stop_words = lines[0].split()

    listed_cont = [[comb.split('/') for comb in sent] for sent in lem_cont]

    main_texts = []
    non_mean = []
    for sent in listed_cont:
        texts = []
        for word in sent:
            # 한 글자 제거
            if len(word[0]) > 1:
                # 조사 등 불필요한 거 제거
                if 'N' in word[-1]:
                    texts.append(word[0])
                elif '@' in word[-1]:
                    texts.append(word[0])
                elif 'V' in word[-1]:
                    texts.append(word[0])
                else:
                    non_mean.append(word[0])
            else:
                non_mean.append(word[0])
        
        main_texts.append(texts)

    lemmatized_question = []

    for noun in main_texts:
        txt = ' '.join(noun)
        lemmatized_question.append(txt)

    merged_df['LEMMA'] = lemmatized_question

    ##Tagset 달기
    tagset = []
    for sent in main_texts:
        tags = []
        for word in sent:
            if '이중전공' == word:
                tags.append("DM")
            elif '부전공' == word:
                tags.append("SM")
            elif '전공심화' == word:
                tags.append("MD")
            elif '융합전공' == word:
                tags.append("UM")
            else:
                tags.append("UK")
        tagset.append(tags)

    tagsets = []
    for sent_tag in tagset:
        tagsets.append(list(set(sent_tag)))

    for tags in tagsets:
        if len(tags) > 1:
            del tags[-1]

    merged_df['TAGSET'] = tagsets

    # csv로 출력
    path = os.getcwd()
    merged_df.to_csv(path + "\\" + result_file_name + ".csv", index=False, encoding="utf-8")

    return merged_df

In [4]:
eta_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문.csv", encoding='ANSI')
eta2_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\질문2.csv", encoding='ANSI')
dm_df = pd.read_csv(r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\이중전과게시판_이중검색.csv", encoding='UTF-8')

merged_df = csv2data(eta_df, eta2_df, dm_df)

In [5]:
space = r"D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\models\soy_big_space_model"
stop = r'D:\Shin\Develop\python\NLP\howls_nlp\Chatbot\자료\stop_words.txt'

preprocess(merged_df, space, stop, "preprocessed_data")

,QUESTION,ANSWER,TAGSET,LEMMA
0,이중전공이랑 부전공 차이에 대해서 궁금해요이중전공은 학위졸업장 표시인건 알겠는데부전...,NaN,"[DM, SM]",이중전공 부전공 차이 대하 이중전공 학위 졸업장 표시 부전공 졸업장 표시
1,재수강 연결방밥 대로 종합정보시스템-수업수강관리-재수강과목선택메뉴를 들어갔는데안내사...,NaN,[UK],재수강 연결 종합 정보 시스템 수업 수강 관리 수강 과목 선택 메뉴 들어가 안내 사...
2,항상 수고가 많으십니다 다름아니라 제가 총평점이 좀 낮은편이라 막학기에 수업을 더 ...,NaN,[UK],수고 다르 아니 제가 평점 학기 수업 평점 올리 졸업 학점 134 학점 초과 문제
3,제가 맨 처음에 이중전공을 gbt로 신청했다가 이중전공 경영으로 변경하려고 시도했는...,NaN,"[SM, DM]",처음 이중전공 신청 이중전공 경영 변경 시도 자체 시험 떨어지 변경 실패 지나 겨울...
4,이중전공을 바꾸고싶은데 혹시 바꾸게 된다면 그전에 받은 학점은 이어가는 건가요 아니...,NaN,[DM],이중전공 바꾸 바꾸 학점 소멸
...,...,...,...,...
6885,중외통 17학번인데요이중전공 어떤 걸로 하면 좋은지 추천해주세요,['아 그렇군요! 감사합니다!!'],[DM],중외 17 학번 이중전공 추천
6886,공대생 이중전공으로 언어할까 하는데 힘듦 언어추천좀,['스페인어 했다가 힘들어서 영어로 바꾼 사람은 봤음'],[DM],공대 이중전공 언어 힘들 언어 추천
6887,국제통상학과 이중전공생인데 걱정이 많이 된다,"['저도요.... 진짜 국통법 때문에 미칠거 같네요 ,', '나두...', '국통 ...",[DM],국제 통상 학과 이중전공 걱정
6888,경영 후기 이중전공 신청할 때이중전공 떨어져서 부전공 경영인데후기 이중전공 경영 신...,"['부전공학점 다 채운다음 이중전공이랑 차이나는 전공점수 채워야되요', '한마디로 ...","[SM, DM]",경영 후기 이중전공 신청 이중전공 떨어지 부전공 경영 후기 이중전공 경영 신청 경영...
